# PSTAT 135 Final Project: Data Preprocessing

In [1]:
# global imports
import pyspark
from pyspark.sql.session import SparkSession
import pyspark.sql.functions as W
from pyspark.sql.types import StringType, ArrayType
# transformations
from pyspark.ml.feature import StringIndexer
# text transformations
import contractions
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from nltk.stem import WordNetLemmatizer

spark = SparkSession.builder.getOrCreate()

# Examine Dataset

In [2]:
# import data
tweets = spark.read.csv('Tweets.csv', header=True, inferSchema=True)
# output dataframe
tweets.show(2)

+------------------+-----------------+----------------------------+--------------+-------------------------+--------------+----------------------+--------+-------------------+-------------+--------------------+-----------+--------------------+--------------+--------------------+
|          tweet_id|airline_sentiment|airline_sentiment_confidence|negativereason|negativereason_confidence|       airline|airline_sentiment_gold|    name|negativereason_gold|retweet_count|                text|tweet_coord|       tweet_created|tweet_location|       user_timezone|
+------------------+-----------------+----------------------------+--------------+-------------------------+--------------+----------------------+--------+-------------------+-------------+--------------------+-----------+--------------------+--------------+--------------------+
|570306133677760513|          neutral|                         1.0|          null|                     null|Virgin America|                  null| cairdin|     

In [3]:
# output dimensions of the dataset
print("Number of rows: ",tweets.count())
print("Number of columns: ",len(tweets.columns))

Number of rows:  14837
Number of columns:  15


In [4]:
# output data types of each column
tweets.dtypes

[('tweet_id', 'string'),
 ('airline_sentiment', 'string'),
 ('airline_sentiment_confidence', 'string'),
 ('negativereason', 'string'),
 ('negativereason_confidence', 'string'),
 ('airline', 'string'),
 ('airline_sentiment_gold', 'string'),
 ('name', 'string'),
 ('negativereason_gold', 'string'),
 ('retweet_count', 'int'),
 ('text', 'string'),
 ('tweet_coord', 'string'),
 ('tweet_created', 'string'),
 ('tweet_location', 'string'),
 ('user_timezone', 'string')]

# Clean Data

## Duplicates

- Drop exact duplicate rows
- Drop rows that differ only by `tweet_id` 

In [5]:
# number of distinct observations
tweets.distinct().count()

14785

In [6]:
# drop duplicates
tweets = tweets.dropDuplicates()
# confirm they were dropped
tweets.count()

14785

In [7]:
# count of distinct rows excluding id
tweets.select([c for c in tweets.columns if c!='tweet_id']).distinct().count()

14751

In [8]:
# output values that are duplicates for all columns except tweet_id
tweets.groupBy('airline_sentiment','airline_sentiment_confidence','negativereason',
               'negativereason_confidence','airline','airline_sentiment_gold',
               'name','negativereason_gold','retweet_count','text','tweet_coord',
               'tweet_created','tweet_location','user_timezone')\
    .count()\
    .where(W.col('count')>1)\
    .show()

+-----------------+----------------------------+--------------------+-------------------------+----------+----------------------+---------------+-------------------+-------------+-----------+-----------+-------------+--------------+-------------+-----+
|airline_sentiment|airline_sentiment_confidence|      negativereason|negativereason_confidence|   airline|airline_sentiment_gold|           name|negativereason_gold|retweet_count|       text|tweet_coord|tweet_created|tweet_location|user_timezone|count|
+-----------------+----------------------------+--------------------+-------------------------+----------+----------------------+---------------+-------------------+-------------+-----------+-----------+-------------+--------------+-------------+-----+
|             null|                        null|                null|                     null|      null|                  null|           null|               null|         null|       null|       null|         null|          null|         

In [9]:
# drop duplicates for rows excluding id
tweets = tweets.dropDuplicates(subset=[c for c in tweets.columns if c!='tweet_id'])
# confirm duplicates were dropped
tweets.count()

14751

## Missing Values

- Remove all rows with missing values for `text`
- Ignore missing values later for columns used in exploratory data analysis

In [10]:
# number of missing values for each column
tweets.select(*(W.sum(W.col(c).isNull().cast('int')).alias(c) for c in tweets.columns))\
      .show()

+--------+-----------------+----------------------------+--------------+-------------------------+-------+----------------------+----+-------------------+-------------+----+-----------+-------------+--------------+-------------+
|tweet_id|airline_sentiment|airline_sentiment_confidence|negativereason|negativereason_confidence|airline|airline_sentiment_gold|name|negativereason_gold|retweet_count|text|tweet_coord|tweet_created|tweet_location|user_timezone|
+--------+-----------------+----------------------------+--------------+-------------------------+-------+----------------------+----+-------------------+-------------+----+-----------+-------------+--------------+-------------+
|       0|              107|                          21|          5508|                     4164|    131|                 14702| 147|              14719|          156| 156|      13686|          338|          4949|         5046|
+--------+-----------------+----------------------------+--------------+------------

In [11]:
# remove rows where text is missing
tweets = tweets.dropna(subset='text')

## Examine Categorical Data

`airline_sentiment`
- change `neutral` to `positive` to create a binary classification problem

In [12]:
# output unique values for each categorical variable
cat_vars = ['airline_sentiment','negativereason','airline','tweet_location','user_timezone']
for col in cat_vars:
    tweets.select(col).distinct().show(20,truncate=False)

+-----------------+
|airline_sentiment|
+-----------------+
|positive         |
|neutral          |
|negative         |
+-----------------+

+---------------------------+
|negativereason             |
+---------------------------+
|Lost Luggage               |
|longlines                  |
|Late Flight                |
|null                       |
|Damaged Luggage            |
|Cancelled Flight           |
|Customer Service Issue     |
|Flight Attendant Complaints|
|Bad Flight                 |
|Can't Tell                 |
|Flight Booking Problems    |
+---------------------------+

+--------------+
|airline       |
+--------------+
|Delta         |
|Virgin America|
|United        |
|US Airways    |
|Southwest     |
|American      |
+--------------+

+--------------------------+
|tweet_location            |
+--------------------------+
|washington, dc            |
|✖️ || 4/5 || ✖️           |
|Winchester | Northend Ma  |
|Heart of America          |
|Bangalore                 |
|Gone

In [13]:
# convert neutral airline tweets to positive
tweets = tweets.withColumn('label', W.when(W.col('airline_sentiment')=='neutral','positive')\
                                          .otherwise(W.col('airline_sentiment')))

## Create Index Column

Used later when combining data frames

In [14]:
# create index column
tweets = tweets.withColumn('id', W.monotonically_increasing_id())
# output data frame
tweets.show(2)

+------------------+-----------------+----------------------------+--------------+-------------------------+--------+----------------------+-----------+-------------------+-------------+--------------------+-----------+--------------------+--------------+-------------+--------+---+
|          tweet_id|airline_sentiment|airline_sentiment_confidence|negativereason|negativereason_confidence| airline|airline_sentiment_gold|       name|negativereason_gold|retweet_count|                text|tweet_coord|       tweet_created|tweet_location|user_timezone|   label| id|
+------------------+-----------------+----------------------------+--------------+-------------------------+--------+----------------------+-----------+-------------------+-------------+--------------------+-----------+--------------------+--------------+-------------+--------+---+
|569912710848057344|         negative|                         1.0|   Late Flight|                      1.0|  United|                  null|ADanaiBaker

## Select Desired Columns

Columns will be used later for exploratory data analysis

In [15]:
# select columns
tweets = tweets.select('id','label','negativereason','airline','retweet_count','text')
# output dataframe
tweets.show(2)

+---+--------+--------------------+----------+-------------+--------------------+
| id|   label|      negativereason|   airline|retweet_count|                text|
+---+--------+--------------------+----------+-------------+--------------------+
|  0|negative|Flight Booking Pr...|US Airways|            0|@USAirways but wa...|
|  1|negative|          Bad Flight|US Airways|            0|@USAirways yes.  ...|
+---+--------+--------------------+----------+-------------+--------------------+
only showing top 2 rows



In [16]:
# output dimensions of the dataset
print("Number of rows: ",tweets.count())
print("Number of columns: ",len(tweets.columns))

Number of rows:  14595
Number of columns:  6


# Transform Text Data

In [17]:
# select columns
text_df = tweets.select('id','label','text')
# output dataframe
text_df.show(2)

+---+--------+--------------------+
| id|   label|                text|
+---+--------+--------------------+
|  0|negative|@USAirways but wa...|
|  1|negative|@USAirways yes.  ...|
+---+--------+--------------------+
only showing top 2 rows



## Expand Contractions

In [18]:
# function for expanding contractions
def fix_contractions(text):
    return contractions.fix(text)
# udf for expanding contractions
contractions_udf = W.udf(lambda row: fix_contractions(row) , StringType())
# add column with contractions expanded
text_df = text_df.withColumn('text_clean', contractions_udf('text'))
# output data frame
text_df.show(2)

+---+--------+--------------------+--------------------+
| id|   label|                text|          text_clean|
+---+--------+--------------------+--------------------+
|  0|negative|@USAirways but wa...|@USAirways but wa...|
|  1|negative|@USAirways yes.  ...|@USAirways yes.  ...|
+---+--------+--------------------+--------------------+
only showing top 2 rows



## Tokenize the Text

In [19]:
# tokenize the text, @():;,.!?\-\/"
rt = RegexTokenizer().setInputCol('text_clean')\
                     .setOutputCol('text_vec')\
                     .setPattern('\s+|[\W]')\
                     .setToLowercase(True)
# transform data
text_df = rt.transform(text_df)
# output dataframe
text_df.show(2)

+---+--------+--------------------+--------------------+--------------------+
| id|   label|                text|          text_clean|            text_vec|
+---+--------+--------------------+--------------------+--------------------+
|  0|negative|@USAirways but wa...|@USAirways but wa...|[usairways, but, ...|
|  1|negative|@USAirways yes.  ...|@USAirways yes.  ...|[usairways, yes, ...|
+---+--------+--------------------+--------------------+--------------------+
only showing top 2 rows



## Convert Digits to Words

Convert `0` through `9` to `zero` through `9`

In [20]:
# dictionary of number and word keys
num_word_dict = {'0':'zero','1':'one','2':'two','3':'three','4':'four',
                 '5':'five','6':'six','7':'seven','8':'eight','9':'nine'}
# function for converting number to word
def num_to_word(row):
    new_row = []
    for x in row:
        if x in num_word_dict.keys():
            new_row.append(num_word_dict[x])
        else:
            new_row.append(x)
    return new_row
# udf for converting number to word
num_to_word_udf = W.udf(lambda row: num_to_word(row) , ArrayType(StringType()))
# add column with numbers converted to word
text_df = text_df.withColumn('text_vec_num', num_to_word_udf('text_vec'))
# output dataframe
text_df.show(2)

+---+--------+--------------------+--------------------+--------------------+--------------------+
| id|   label|                text|          text_clean|            text_vec|        text_vec_num|
+---+--------+--------------------+--------------------+--------------------+--------------------+
|  0|negative|@USAirways but wa...|@USAirways but wa...|[usairways, but, ...|[usairways, but, ...|
|  1|negative|@USAirways yes.  ...|@USAirways yes.  ...|[usairways, yes, ...|[usairways, yes, ...|
+---+--------+--------------------+--------------------+--------------------+--------------------+
only showing top 2 rows



## Remove Stopwords

In [21]:
# create english stopwords
english = StopWordsRemover().loadDefaultStopWords('english')
# stopwords transformer
stops = StopWordsRemover().setStopWords(english)\
                          .setInputCol('text_vec_num')\
                          .setOutputCol('text_vec_stop')
# transform dataframe
text_df = stops.transform(text_df)
# output dataframe
text_df.show(2)

+---+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
| id|   label|                text|          text_clean|            text_vec|        text_vec_num|       text_vec_stop|
+---+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  0|negative|@USAirways but wa...|@USAirways but wa...|[usairways, but, ...|[usairways, but, ...|[usairways, wait,...|
|  1|negative|@USAirways yes.  ...|@USAirways yes.  ...|[usairways, yes, ...|[usairways, yes, ...|[usairways, yes, ...|
+---+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 2 rows



## Lemmatize Words

In [22]:
# create word lemmatizer object
wnl = WordNetLemmatizer()
# function for lemmatizing words
def wnl_row(row):
    return [wnl.lemmatize(x) for x in row]
# udf for lemmatizing words
lemmatizer_udf = W.udf(lambda row: wnl_row(row) , ArrayType(StringType()))
# create column of lemmatized words
text_df = text_df.withColumn('text_vec_lem', lemmatizer_udf('text_vec_stop'))
# output dataframe
text_df.show(2)

+---+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| id|   label|                text|          text_clean|            text_vec|        text_vec_num|       text_vec_stop|        text_vec_lem|
+---+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  0|negative|@USAirways but wa...|@USAirways but wa...|[usairways, but, ...|[usairways, but, ...|[usairways, wait,...|[usairways, wait,...|
|  1|negative|@USAirways yes.  ...|@USAirways yes.  ...|[usairways, yes, ...|[usairways, yes, ...|[usairways, yes, ...|[usairways, yes, ...|
+---+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 2 rows



## Remove Characters

In [23]:
# function for removing characters
def not_letter(row):
    return [x for x in row if len(x)>1]
# udf for removing characters
not_letter_udf = W.udf(lambda row: not_letter(row) , ArrayType(StringType()))
# create column with characters removed
text_df = text_df.withColumn('text_vec_clean', not_letter_udf('text_vec_lem'))
# output dataframe
text_df.show(2)

+---+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| id|   label|                text|          text_clean|            text_vec|        text_vec_num|       text_vec_stop|        text_vec_lem|      text_vec_clean|
+---+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  0|negative|@USAirways but wa...|@USAirways but wa...|[usairways, but, ...|[usairways, but, ...|[usairways, wait,...|[usairways, wait,...|[usairways, wait,...|
|  1|negative|@USAirways yes.  ...|@USAirways yes.  ...|[usairways, yes, ...|[usairways, yes, ...|[usairways, yes, ...|[usairways, yes, ...|[usairways, yes, ...|
+---+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 2 rows



## Vectorize Words

Create a vocabulary of words that appear in 0.5% of the documents

In [24]:
# create transformer
cv = CountVectorizer().setInputCol('text_vec_clean')\
                      .setOutputCol('count_vec')\
                      .setMinDF(text_df.count()*0.005)
# fit the transformer
fit_cv = cv.fit(text_df)
# transform the data
text_df = fit_cv.transform(text_df)
# output the vectorized column
text_df.select('count_vec').show(5, truncate=False)

+-------------------------------------------------------------------------------+
|count_vec                                                                      |
+-------------------------------------------------------------------------------+
|(349,[2,58,118,127,152],[1.0,1.0,1.0,1.0,2.0])                                 |
|(349,[0,2,16,22,40,96,210,286],[1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0])              |
|(349,[0,4,134,186,285],[1.0,1.0,1.0,1.0,1.0])                                  |
|(349,[5,7,8,158,159],[1.0,1.0,1.0,1.0,1.0])                                    |
|(349,[0,5,6,24,54,70,81,114,275,321],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
+-------------------------------------------------------------------------------+
only showing top 5 rows



In [25]:
# create vocabulary variable
vocab = fit_cv.vocabulary

## Convert Label Column to Float

`negative` becomes `0` and `positive` becomes `1`

In [26]:
# create string indexer
indxr = StringIndexer().setInputCol('label')\
                       .setOutputCol('label_idx')
# transform data
text_df = indxr.fit(text_df).transform(text_df)
# output dataframe
text_df.show(2)

+---+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+
| id|   label|                text|          text_clean|            text_vec|        text_vec_num|       text_vec_stop|        text_vec_lem|      text_vec_clean|           count_vec|label_idx|
+---+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+
|  0|negative|@USAirways but wa...|@USAirways but wa...|[usairways, but, ...|[usairways, but, ...|[usairways, wait,...|[usairways, wait,...|[usairways, wait,...|(349,[2,58,118,12...|      0.0|
|  1|negative|@USAirways yes.  ...|@USAirways yes.  ...|[usairways, yes, ...|[usairways, yes, ...|[usairways, yes, ...|[usairways, yes, ...|[usairways, yes, ...|(349,[0,2,16,22,4...|      0.0|
+---+--------+--------------------+

# Train and Test Datasets

70% of the data is used to train the model, 30% of the data is used to test the model

In [27]:
# create train and test sets
train,test = text_df.randomSplit([0.7,0.3],116)

In [28]:
# output train data
train.show(2)

+---+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+
| id|   label|                text|          text_clean|            text_vec|        text_vec_num|       text_vec_stop|        text_vec_lem|      text_vec_clean|           count_vec|label_idx|
+---+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+
|  1|negative|@USAirways yes.  ...|@USAirways yes.  ...|[usairways, yes, ...|[usairways, yes, ...|[usairways, yes, ...|[usairways, yes, ...|[usairways, yes, ...|(349,[0,2,16,22,4...|      0.0|
|  3|positive|Sigh... “@JetBlue...|Sigh... “@JetBlue...|[sigh, jetblue, o...|[sigh, jetblue, o...|[sigh, jetblue, f...|[sigh, jetblue, f...|[sigh, jetblue, f...|(349,[5,7,8,158,1...|      1.0|
+---+--------+--------------------+

## Tweets Dataset for Exploratory Data Analysis

Make a dataset for the training data with additional features for exploratory data analysis

In [29]:
# merge tweets and train to get a data frame for exploratory data analysis with additional features
tweets_train = tweets.join(train, on=['id'], how='left_semi')
tweets_train = tweets_train.select('label','negativereason','airline','retweet_count','text')
tweets_train.show(2)

+--------+--------------+----------+-------------+--------------------+
|   label|negativereason|   airline|retweet_count|                text|
+--------+--------------+----------+-------------+--------------------+
|negative|    Bad Flight|US Airways|            0|@USAirways yes.  ...|
|positive|          null|     Delta|            0|Sigh... “@JetBlue...|
+--------+--------------+----------+-------------+--------------------+
only showing top 2 rows



## Select Columns for Train and Test

In [30]:
# select columns for train and test
train = train.select('count_vec','label_idx')
test = test.select('count_vec','label_idx')

## Store Data Frames for Later Use

In [31]:
# store dataframes for additional programs
dfs = [train.toPandas(),test.toPandas(),tweets_train.toPandas(),vocab]
%store dfs

Stored 'dfs' (list)
